In [ ]:
import torch
import cv2
import pandas as pd
import numpy as np

from twilio.rest import Client

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

In [ ]:
video_path = 'IMG_3492.mov'
output_path = 'output.mp4'

In [ ]:
def alert():

    account_sid = 'your_account_sid'
    auth_token = 'your_auth_token'
    twilio_number = 'whatsapp:twilio_number'
    recipient_number = 'whatsapp:recipient_number'
    
    client = Client(account_sid, auth_token)
    
    client.messages.create(body='Alert: Get your Kid!', 
                          from_=twilio_number, 
                          to=recipient_number)

In [ ]:
def process_video(video_path, output_path="output_video.mp4", alert_threshold=1):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    # Define the video writer for output video
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Run YOLO detection on the current frame
        results = model(frame)
        detections = results.pandas().xyxy[0]  # YOLO results as a DataFrame

        # Count detected persons
        children_count = 0
        for _, row in detections.iterrows():
            if row['name'] == 'person':
                children_count += 1

        # Draw bounding boxes and labels on the frame
        for _, row in detections.iterrows():
            if row['name'] == 'person':
                color = (0, 255, 0)  # Color for detected persons
                cv2.rectangle(frame, (int(row['xmin']), int(row['ymin'])), (int(row['xmax']), int(row['ymax'])), color, 2)
                cv2.putText(frame, f"{row['name']}", (int(row['xmin']), int(row['ymin']) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        # Display the children count and trigger alert if below threshold
        cv2.putText(frame, f"Children count: {children_count}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        if children_count < alert_threshold:
            cv2.putText(frame, "ALERT: Low children count!", (10, 70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            alert()  # Call the alert function
            print("Alert! Only one or two children remain in the frame.")
            break
        
        # Write frame to output video
        out.write(frame)

        # Display the frame (optional)
        cv2.imshow("Surveillance", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release video objects and close OpenCV windows
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [ ]:
process_video(video_path, output_path)